In [12]:
import pandas as pd
import requests

baseUrl = 'https://browser.ihtsdotools.org/snowstorm/snomed-ct/v2/'
edition = 'MAIN'
version = '2019-07-31'
sctid = '109152007'
uri = baseUrl + 'browser/' + edition + '/' + version + '/concepts/' + sctid

In [13]:
resp = requests.get(uri)
resp

<Response [200]>

In [17]:
resp.json()['fsn']['term']

'Bilirubin test kit (physical object)'